# Merge overlapping polygons

I have two polygon shapefiles
1. Villages of Assam.
2. Revenue Circles of Assam.

I want to merge them both and find out in which revenue circle the village falls in. 

BUT - a few villages cut across multiple revenue circles - The [maze of boundaries](https://www.thehindu.com/opinion/op-ed/redraw-the-lines-for-better-planning/article19235528.ece) is a national problem.


In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
rcs_gdf = gpd.read_file('assam_revenue_circle_nov2022.geojson',driver='GeoJSON')
rcs_gdf.tail()
#180 rows = 180 rcs

,object_id,revenue_ci,geometry
175,278,Sapekhati,"MULTIPOLYGON (((95.10936 27.12364, 95.10927 27..."
176,279,Sonari,"MULTIPOLYGON (((94.95742 27.06354, 94.95822 27..."
177,280,Ujani Majuli,"MULTIPOLYGON (((94.33804 26.89186, 94.33850 26..."
178,281,Majuli,"MULTIPOLYGON (((94.56453 27.18172, 94.56755 27..."
179,275,Phuloni,"MULTIPOLYGON (((92.88407 26.38783, 92.88894 26..."


In [3]:
districts_gdf = gpd.read_file('assam_district_35.geojson', driver='GeoJSON')
districts_gdf = districts_gdf.to_crs(rcs_gdf.crs)
districts_gdf.tail()

,state,district,ID,assam_dist,geometry
30,ASSAM,SIVSAGAR,31,2011Siva,"MULTIPOLYGON (((94.67453 26.72628, 94.67252 26..."
31,ASSAM,BARPETA,32,2011Barp,"MULTIPOLYGON (((90.99640 26.58425, 90.99685 26..."
32,ASSAM,BAJALI,33,2022Baja,"MULTIPOLYGON (((91.23737 26.58898, 91.23737 26..."
33,ASSAM,TAMULPUR,34,2011Tamu,"MULTIPOLYGON (((91.41287 26.83627, 91.41514 26..."
34,ASSAM,BAKSA,35,2011Baks,"MULTIPOLYGON (((91.14036 26.65999, 91.13993 26..."


Let's do a simple spatial join and see what happens

In [4]:
merged_gdf = gpd.sjoin(rcs_gdf,
                       districts_gdf,
                       how="left",
                       op="intersects")

/home/krishna/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
merged_gdf

,object_id,revenue_ci,geometry,index_right,state,district,ID,assam_dist
0,101,Gossaigaon (Pt),"MULTIPOLYGON (((90.14118 26.74010, 90.15342 26...",0,ASSAM,DHUBRI,1,2011Dhub
0,101,Gossaigaon (Pt),"MULTIPOLYGON (((90.14118 26.74010, 90.15342 26...",8,ASSAM,KOKRAJHAR,9,2011Kokr
1,102,Bhowraguri,"MULTIPOLYGON (((90.09811 26.46455, 90.10012 26...",8,ASSAM,KOKRAJHAR,9,2011Kokr
2,103,Dotoma,"MULTIPOLYGON (((90.19819 26.63409, 90.21635 26...",0,ASSAM,DHUBRI,1,2011Dhub
2,103,Dotoma,"MULTIPOLYGON (((90.19819 26.63409, 90.21635 26...",8,ASSAM,KOKRAJHAR,9,2011Kokr
...,...,...,...,...,...,...,...,...
178,281,Majuli,"MULTIPOLYGON (((94.56453 27.18172, 94.56755 27...",26,ASSAM,DIBRUGARH,27,2011Dibr
179,275,Phuloni,"MULTIPOLYGON (((92.88407 26.38783, 92.88894 26...",22,ASSAM,HOJAI,23,2019Hoja
179,275,Phuloni,"MULTIPOLYGON (((92.88407 26.38783, 92.88894 26...",7,ASSAM,K.ANGLONG,8,2011Karb
179,275,Phuloni,"MULTIPOLYGON (((92.88407 26.38783, 92.88894 26...",10,ASSAM,NAGAON,11,2011Naga


Whoops! 180 rows exploded and became 485. Revenue circles cut across multiple districts.

1. `Gossaigaon (Pt)` falls in both `DHUBRI` and `KOKRAJHAR`
2. `Phuloni` falls in `HOJAI`, `K.ANGLONG`, `NAGAON` and `GOLAGHAT`

etc.

Say I want to know which District Collector is responsible for a revenue circle. And if I belong to Phuloni, I have to trot between 4 District collectors. 

**To avoid this, I want to find the district which has a maximum intersection with my revenue circle.**

In [7]:
intersection_result = gpd.overlay(rcs_gdf, districts_gdf, how = 'intersection',keep_geom_type=False)
intersection_result

,object_id,revenue_ci,state,district,ID,assam_dist,geometry
0,101,Gossaigaon (Pt),ASSAM,DHUBRI,1,2011Dhub,"MULTIPOLYGON (((89.90561 26.32791, 89.90555 26..."
1,103,Dotoma,ASSAM,DHUBRI,1,2011Dhub,GEOMETRYCOLLECTION (POLYGON ((90.20512 26.3711...
2,104,Kokrajhar (Pt),ASSAM,DHUBRI,1,2011Dhub,GEOMETRYCOLLECTION (POLYGON ((90.42501 26.3630...
3,105,Bagribari (Pt),ASSAM,DHUBRI,1,2011Dhub,GEOMETRYCOLLECTION (POLYGON ((90.11704 26.3379...
4,106,Golokganj (Pt),ASSAM,DHUBRI,1,2011Dhub,GEOMETRYCOLLECTION (POLYGON ((89.94024 26.1256...
...,...,...,...,...,...,...,...
480,267,North Guwahati (Pt),ASSAM,KAMRUP METRO,29,2011KamrM,"POLYGON ((91.68052 26.21425, 91.67951 26.21428..."
481,269,Mangaldoi (Pt),ASSAM,KAMRUP METRO,29,2011KamrM,"MULTILINESTRING ((91.97864 26.27422, 91.97862 ..."
482,271,Mayong,ASSAM,KAMRUP METRO,29,2011KamrM,GEOMETRYCOLLECTION (POLYGON ((92.15768 26.1694...
483,272,Palasbari,ASSAM,KAMRUP METRO,29,2011KamrM,GEOMETRYCOLLECTION (POLYGON ((91.56155 26.1246...


Now I want to select the distirct which shares maximum intersection area with the revenue circle.

In [8]:
intersection_result['intersection_area'] = intersection_result.geometry.area

/tmp/ipykernel_306842/763243640.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersection_result['intersection_area'] = intersection_result.geometry.area


In [9]:
max_indices = intersection_result.groupby(['object_id'])['intersection_area'].idxmax()

In [10]:
intersection_result = intersection_result.loc[max_indices]

In [11]:
intersection_result

,object_id,revenue_ci,state,district,ID,assam_dist,geometry,intersection_area
18,101,Gossaigaon (Pt),ASSAM,KOKRAJHAR,9,2011Kokr,"POLYGON ((90.15342 26.72894, 90.17980 26.70689...",0.096800
19,102,Bhowraguri,ASSAM,KOKRAJHAR,9,2011Kokr,"POLYGON ((90.10012 26.46386, 90.10036 26.46374...",0.014371
20,103,Dotoma,ASSAM,KOKRAJHAR,9,2011Kokr,"POLYGON ((90.21635 26.62661, 90.21760 26.62614...",0.027516
21,104,Kokrajhar (Pt),ASSAM,KOKRAJHAR,9,2011Kokr,"POLYGON ((90.33610 26.85565, 90.33624 26.85077...",0.089722
22,105,Bagribari (Pt),ASSAM,KOKRAJHAR,9,2011Kokr,"POLYGON ((89.99717 26.34893, 89.99723 26.34880...",0.025390
...,...,...,...,...,...,...,...,...
418,277,Jorhat East,ASSAM,JORHAT,22,2011Jorh,"POLYGON ((94.22535 26.73834, 94.22461 26.74109...",0.017743
407,278,Sapekhati,ASSAM,CHARAIDEO,5,2019Char,"POLYGON ((95.10927 27.12542, 95.10847 27.12662...",0.035886
408,279,Sonari,ASSAM,CHARAIDEO,5,2019Char,"POLYGON ((94.95822 27.06434, 94.95955 27.06458...",0.034517
349,280,Ujani Majuli,ASSAM,MAJULI,18,2019Maju,"MULTIPOLYGON (((94.24115 27.02868, 94.23823 27...",0.029304


In [12]:
intersection_result[intersection_result['revenue_ci']=='Phuloni']

,object_id,revenue_ci,state,district,ID,assam_dist,geometry,intersection_area
288,275,Phuloni,ASSAM,K.ANGLONG,8,2011Karb,"POLYGON ((92.88892 26.38828, 92.88908 26.38855...",0.297289


In [ ]:
Yay! I ca